In [1]:
import pandas as pd
import os
from itertools import chain
import altair as alt
import numpy as np
import ast

notebook_dir = os.getcwd()
results_path = os.path.normpath(os.path.join(notebook_dir, "outputs", "updated_model_results.csv"))
norms_path   = os.path.normpath(os.path.join(notebook_dir, "data", "all_8bit_norms_with_dnf.csv"))

# Load CSVs
results_df = pd.read_csv(results_path)
norms_df   = pd.read_csv(norms_path, dtype={"8bit_vector": str})

# --- Helpers to flatten ---
def flatten_ebsn_to_str(ebsn):
    flat_list = list(chain.from_iterable(chain.from_iterable(ebsn)))
    return ''.join(str(int(b)) for b in flat_list)

def flatten_base_sn_to_str(base_sn):
    flat_list = list(chain.from_iterable(base_sn))
    return ''.join(str(int(b)) for b in flat_list)


def identify_base_norm(base_norm_str: str) -> str:
    """
    Identify the base social norm (e.g. Image Scoring, Stern Judging, etc.)
    from its 4-bit structure [[a,b], [c,d], ...] as stored in the dataframe.
    """
    try:
        norm = ast.literal_eval(base_norm_str)
    except Exception:
        return "Unknown"

    # Flatten if nested
    flat = [int(x) for pair in norm for x in pair]

    mapping = {
        (0, 0, 1, 1): "Image Scoring",
        (1, 0, 0, 1): "Stern Judging",
        (0, 0, 0, 1): "Shunning",
        (1, 0, 1, 1): "Simple Standing",
        (0, 0, 0, 0): "All Bad",
        (1, 1, 1, 1): "All Good",
    }

    return mapping.get(tuple(flat), "Unknown")


# Flatten columns in results
results_df['8bit_vector'] = results_df['eb_social_norm'].apply(eval).apply(flatten_ebsn_to_str)
results_df['4bit_orig']   = results_df['base_social_norm'].apply(eval).apply(flatten_base_sn_to_str)

# Merge and include DNF columns
merged_df = pd.merge(
    results_df,
    norms_df[["8bit_vector", 
              "Emotion_Leniency", "DNF", "DNF_literals"]],
    on=["8bit_vector"],
    how="left"
)

# Ensure numeric
merged_df["DNF_literals"] = pd.to_numeric(merged_df["DNF_literals"], errors="coerce")
merged_df["base_social_norm"] = merged_df["base_social_norm"].apply(identify_base_norm)


In [2]:
merged_df

,base_social_norm,eb_social_norm,Z,gens,mu,chi,eps,alpha,b,c,...,DISCRIMINATE,PARADOXICALLY_DISC,ALWAYS_DEFECT,Competitive,Cooperative,8bit_vector,4bit_orig,Emotion_Leniency,DNF,DNF_literals
0,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.5333,0.1333,0.3333,0.87,0.13,00011111,0011,0.75,A | (E & R),3
1,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.1333,0.1333,0.7333,0.73,0.27,00011111,0011,0.75,A | (E & R),3
2,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.0000,0.0333,0.9333,0.07,0.93,00011111,0011,0.75,A | (E & R),3
3,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.2333,0.0000,0.4000,0.60,0.40,00011111,0011,0.75,A | (E & R),3
4,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.0333,0.0000,0.9333,0.93,0.07,00011111,0011,0.75,A | (E & R),3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11543,Stern Judging,"[[(1, 1), (1, 1)], [(1, 1), (1, 1)]]",50,1000,1.0,0.01,0.01,0.0,5,1,...,0.0600,0.1200,0.8200,0.06,0.94,11111111,1001,1.00,True,0
11544,Stern Judging,"[[(1, 1), (1, 1)], [(1, 1), (1, 1)]]",50,1000,1.0,0.01,0.01,0.0,5,1,...,0.0400,0.0000,0.9600,0.04,0.96,11111111,1001,1.00,True,0
11545,Stern Judging,"[[(1, 1), (1, 1)], [(1, 1), (1, 1)]]",50,1000,1.0,0.01,0.01,0.0,5,1,...,0.0000,0.3600,0.6400,0.36,0.64,11111111,1001,1.00,True,0
11546,Stern Judging,"[[(1, 1), (1, 1)], [(1, 1), (1, 1)]]",50,1000,1.0,0.01,0.01,0.0,5,1,...,0.1200,0.0200,0.8600,0.12,0.88,11111111,1001,1.00,True,0


## 📊 Emergent Norms: DNF Complexity vs Cooperation  
This section selects a **base social norm** (e.g., Image Scoring, Stern Judging) and visualizes how all **emergent 8-bit norms** derived from it perform.

For each emergent norm:
- All simulation runs are grouped.
- The **mean cooperation ratio** is computed.
- The **DNF complexity** (number of literals in simplified DNF) is retrieved.

The scatterplot shows:
- **x-axis:** DNF complexity  
- **y-axis:** mean cooperation  
- **each point:** one emergent 8-bit social norm  

This helps reveal which evolved norms are both **simple** and **highly cooperative** under a chosen base norm.


In [34]:
# =============================
#   Scatterplot by Base Norm
# =============================

# 1. Choose base norm (must match identify_base_norm output)
chosen_base_norm = "Stern Judging"   # ← adjust as needed

# Filter only runs that originate from this base norm
filtered = merged_df[merged_df["base_social_norm"] == chosen_base_norm].copy()
filtered = filtered[filtered.Z == 50]
filtered = filtered[filtered.gens == 1000]
filtered = filtered[filtered.gamma_center == 0.8]

# 2. Average all runs per emergent norm
grouped = (
    filtered.groupby(["8bit_vector", "DNF_literals", "Emotion_Leniency"], as_index=False)
            .agg(mean_coop=("average_cooperation", "mean"))
)

# 3. Keep only the top 10% for each DNF complexity
def top_10_percent(df):
    if len(df) == 0:
        return df
    cutoff = np.quantile(df["mean_coop"], 0.90)
    return df[df["mean_coop"] >= cutoff]

grouped_top = (
    grouped.groupby("DNF_literals", group_keys=False)
           .apply(top_10_percent)
)

# Scatter plot with color by Emotion Leniency
scatter = (
    alt.Chart(grouped_top)
    .mark_circle(size=80)
    .encode(
        x=alt.X("DNF_literals:Q", title="DNF Complexity"),
        y=alt.Y("mean_coop:Q", title="Mean Cooperation", scale = alt.Scale(domain=[0,100])),
        color=alt.Color("Emotion_Leniency:Q", title="Emotion Leniency",
                        scale=alt.Scale(scheme="viridis")),
        tooltip=[
            "8bit_vector",
            "DNF_literals",
            "Emotion_Leniency",
            alt.Tooltip("mean_coop:Q", format=".4f")
        ]
    )
)

# Polynomial regression (degree 2)
poly_reg = (
    alt.Chart(grouped_top)
    .transform_regression(
        "DNF_literals",
        "mean_coop",
        method="poly"
    )
    .mark_line(size=2, color="black", opacity=0.5)
    .encode(
        x="DNF_literals:Q",
        y="mean_coop:Q"
    )
)

# Combine
plot = (
    (scatter + poly_reg)
    .properties(
        width=500,
        height=300,
        title=f"Social Norms (Top 10%) — Poly Regression — Base Norm: {chosen_base_norm}"
    )
)

plot


alt.LayerChart(...)

In [33]:
filtered.gamma_center.unique()


array([0.8])

In [11]:
# --- Select a base social norm to examine ---
# Filter to only runs that used this base norm
sub = merged_df[merged_df["base_social_norm"] == chosen_base_norm].copy()

# Group by emergent norm and compute mean cooperation
leniency_summary = (
    sub.groupby(["8bit_vector", "Emotion_Leniency"], as_index=False)
       .agg(mean_coop=("average_cooperation", "mean"))
)

leniency_summary.head()

# Scatter plot
scatter_leniency = (
    alt.Chart(leniency_summary)
       .mark_circle(size=80, opacity=0.8)
       .encode(
           x=alt.X("Emotion_Leniency:Q",
                   title="Emotion Leniency (0 = harsh, 1 = forgiving)"),
           y=alt.Y("mean_coop:Q",
                   title="Mean Cooperation"),
           tooltip=[
               alt.Tooltip("8bit_vector:N", title="Norm vector"),
               alt.Tooltip("Emotion_Leniency:Q", title="Leniency", format=".3f"),
               alt.Tooltip("mean_coop:Q", title="Mean Coop", format=".3f"),
           ]
       )
       .properties(
           width=500,
           height=350,
           title=f"Emotion Leniency vs Cooperation — {chosen_base_norm}"
       )
)

scatter_leniency



alt.Chart(...)

In [4]:
filtered

,base_social_norm,eb_social_norm,Z,gens,mu,chi,eps,alpha,b,c,...,DISCRIMINATE,PARADOXICALLY_DISC,ALWAYS_DEFECT,Competitive,Cooperative,8bit_vector,4bit_orig,Emotion_Leniency,DNF,DNF_literals
0,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.5333,0.1333,0.3333,0.87,0.13,00011111,0011,0.75,A | (E & R),3
1,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.1333,0.1333,0.7333,0.73,0.27,00011111,0011,0.75,A | (E & R),3
2,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.0000,0.0333,0.9333,0.07,0.93,00011111,0011,0.75,A | (E & R),3
3,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.2333,0.0000,0.4000,0.60,0.40,00011111,0011,0.75,A | (E & R),3
4,Image Scoring,"[[(0, 0), (0, 1)], [(1, 1), (1, 1)]]",30,250,1.0,0.01,0.01,0.0,5,1,...,0.0333,0.0000,0.9333,0.93,0.07,00011111,0011,0.75,A | (E & R),3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5143,Image Scoring,"[[(1, 1), (0, 1)], [(1, 0), (1, 0)]]",50,1000,1.0,0.01,0.01,0.0,5,1,...,0.0400,0.0200,0.9200,0.92,0.08,11011010,0011,0.25,(A & ~E) | (E & ~A) | (~A & ~R),6
5144,Image Scoring,"[[(1, 1), (0, 1)], [(1, 0), (1, 0)]]",50,1000,1.0,0.01,0.01,0.0,5,1,...,0.0200,0.0000,0.9800,0.98,0.02,11011010,0011,0.25,(A & ~E) | (E & ~A) | (~A & ~R),6
5145,Image Scoring,"[[(1, 1), (0, 1)], [(1, 0), (1, 0)]]",50,1000,1.0,0.01,0.01,0.0,5,1,...,0.0000,0.0400,0.9600,0.96,0.04,11011010,0011,0.25,(A & ~E) | (E & ~A) | (~A & ~R),6
5146,Image Scoring,"[[(1, 1), (0, 1)], [(1, 0), (1, 0)]]",50,1000,1.0,0.01,0.01,0.0,5,1,...,0.0000,0.0000,1.0000,1.00,0.00,11011010,0011,0.25,(A & ~E) | (E & ~A) | (~A & ~R),6


## Multiple Line Charts

In [38]:
# Pick norm
chosen_norm = "SternJudging"
norm_df = merged_df[merged_df.norm == chosen_norm].copy()
norm_df["is_base"] = norm_df["variant_id"].str.endswith("_v1")

# If your results use a different name (e.g., "gamma_gaussian_n"),
# rename it once so plots are consistent:
if "gamma_center" not in norm_df.columns and "gamma_gaussian_n" in norm_df.columns:
    norm_df = norm_df.rename(columns={"gamma_gaussian_n": "gamma_center"})

# Aggregate per variant & gamma, carry DNF info (constant per variant)
agg_df = (
    norm_df
    .groupby(["variant_id", "DNF_literals", "Emotion_Leniency", "gamma_center", "is_base"], as_index=False)
    .agg(
        avg_coop=("average_cooperation", "mean"),
        std_coop=("average_cooperation", "std"),
        DNF=("DNF", "first")
    )
    .sort_values(["variant_id", "gamma_center"])
)

# Make a percent-friendly copy
plot_df = agg_df.copy()

# If avg_coop is in [0,1], convert to %; if already 0–100, keep as-is
def to_percent(col):
    arr = col.to_numpy(dtype=float)
    # heuristic: if most values ≤ 1, treat as proportions
    needs_scale = (np.nanmean(arr <= 1.0) > 0.5)
    return arr * 100.0 if needs_scale else arr

plot_df["avg_coop_percent"] = to_percent(plot_df["avg_coop"])
plot_df["std_coop_percent"] = to_percent(plot_df["std_coop"])

# Base lines (no selections; color by DNF_literals; dashed if base)
line = alt.Chart(plot_df).mark_line().encode(
    x=alt.X("gamma_center:Q", title="Gamma value"),
    y=alt.Y("avg_coop_percent:Q",
            title="Average Cooperation (%)",
            scale=alt.Scale(domain=[0, 100])),
    color=alt.Color("Emotion_Leniency:O",
                    title="Emotion Leniency",
                    scale=alt.Scale(scheme="bluepurple")),
    strokeDash=alt.condition(
        alt.datum.is_base,
        alt.value([5, 5]),    # dashed for base
        alt.value([1, 0])     # solid otherwise
    ),
    strokeWidth=alt.condition(
        alt.datum.is_base,
        alt.value(8),
        alt.value(2)
    ),
    tooltip=[
        alt.Tooltip("variant_id:N", title="Variant"),
        alt.Tooltip("gamma_center:Q", title="Gamma"),
        alt.Tooltip("avg_coop_percent:Q", title="Mean coop (%)", format=".1f"),
        alt.Tooltip("std_coop_percent:Q", title="Std (%)", format=".1f"),
        alt.Tooltip("Emotion_Leniency:O", title="Leniency"),
        alt.Tooltip("is_base:N", title="Base?")
    ],
    detail="variant_id:N"
)

# Optional: end-of-line labels (still no interactivity)
endpoints = (
    plot_df.sort_values(["variant_id", "gamma_center"])
           .groupby("variant_id", as_index=False)
           .tail(1)
)

labels = alt.Chart(endpoints).mark_text(
    dx=5, align="left", baseline="middle"
).encode(
    x="gamma_center:Q",
    y=alt.Y("avg_coop_percent:Q",
            scale=alt.Scale(domain=[0, 100])),
    text="variant_id:N"
)

chart = (line + labels).properties(
    width=700, height=420,
    title=f"Performance of {chosen_norm} Variants (color = Emotion Leniency; dashed = base)"
)

chart

alt.LayerChart(...)

In [39]:

# Pivot: rows = variant_id, cols = gamma_center
pivot_df = plot_df.pivot_table(
    index="variant_id",
    columns="gamma_center",
    values="avg_coop_percent",
    aggfunc="mean"   # though already averaged
).reset_index()

# Optional: rename gamma columns for clarity (e.g., "gamma 0.0")
pivot_df = pivot_df.rename(
    columns={g: f"gamma {g}" for g in pivot_df.columns if isinstance(g, (int, float))}
)

# Check result
pivot_df

gamma_center,variant_id,gamma 0.0,gamma 0.1,gamma 0.2,gamma 0.3,gamma 0.4,gamma 0.5,gamma 0.6,gamma 0.7,gamma 0.8,gamma 0.9,gamma 1.0
0,Stern_Judging_v1,93.28864,93.26890,93.47752,92.83250,92.90152,93.19750,93.09698,93.25764,93.08008,93.04866,93.34372
1,Stern_Judging_v10,93.33162,92.14812,90.55076,88.20090,83.78480,79.97648,80.33770,73.20560,62.97154,27.80382,21.30126
2,Stern_Judging_v11,93.35224,92.55862,92.17434,90.76350,92.16544,92.27366,92.02842,92.09494,91.76820,90.48394,90.72412
3,Stern_Judging_v12,93.14702,91.59006,90.24360,89.42606,85.20590,84.27042,86.59692,81.48238,73.90780,47.14592,36.66792
4,Stern_Judging_v13,93.28780,92.97808,93.00762,92.03416,90.85938,85.99830,71.79606,23.99790,6.45368,5.61582,5.41884
5,Stern_Judging_v14,92.87316,91.79338,89.94936,88.82904,85.59362,72.08164,57.18136,24.64146,6.17852,5.80872,5.46212
6,Stern_Judging_v15,92.76920,92.70026,90.73910,87.63084,81.96890,77.00726,53.06528,26.19612,7.03662,5.64558,5.38878
7,Stern_Judging_v16,93.11822,91.25736,90.86474,80.59694,72.66954,51.16400,26.50390,10.14328,6.17164,5.48934,5.38526
8,Stern_Judging_v2,93.03454,92.67144,92.13582,91.89320,91.25692,90.92680,89.79384,91.10638,90.40572,90.30824,90.58912
9,Stern_Judging_v3,93.39326,92.60914,91.73056,91.41912,91.90674,91.41670,91.62090,90.92022,86.69258,90.44260,91.12550


# HEATMAPS
## Complexity

In [75]:
import numpy as np
import pandas as pd
import altair as alt

chosen_norm = "SternJudging"

# Filter to chosen norm
norm_df = merged_df[merged_df.norm == chosen_norm].copy()

# Ensure consistent gamma column name
if "gamma_center" not in norm_df.columns and "gamma_gaussian_n" in norm_df.columns:
    norm_df = norm_df.rename(columns={"gamma_gaussian_n": "gamma_center"})

# Round γ and keep only clean bins (0.0, 0.1, ..., 1.0)
valid_gammas = np.round(np.arange(0, 1.01, 0.1), 1)
norm_df = norm_df[norm_df["gamma_center"].isin(valid_gammas)]

# ---- STEP 1: average cooperation per variant (across runs) ----
variant_avg = (
    norm_df
    .groupby(["norm", "variant_id", "gamma_center", "DNF_literals"], as_index=False)
    .agg(mean_coop=("average_cooperation", "mean"))
)

# ---- STEP 2: for each (γ, DNF_literals), take the variant with the highest average ----
variant_avg_sorted = variant_avg.sort_values("mean_coop", ascending=False)
agg = (
    variant_avg_sorted
    .groupby(["gamma_center", "DNF_literals"], as_index=False)
    .agg(
        max_mean_coop=("mean_coop", "max"),
        best_variant=("variant_id", "first"),
        n=("variant_id", "nunique")
    )
)

agg["DNF_literals"] = agg["DNF_literals"].astype(int)

# Convert to percentage if needed
if agg["max_mean_coop"].max() <= 1:
    agg["max_mean_coop"] *= 100

# ---- STEP 3: Visualization ----
heat = alt.Chart(agg).mark_rect().encode(
    x=alt.X("gamma_center:O", title="γ",
            sort=[f"{x:.1f}" for x in valid_gammas]),
    y=alt.Y("DNF_literals:O", title="DNF literals", sort="descending"),
    color=alt.Color("max_mean_coop:Q", title="Max mean cooperation (%)",
                    scale=alt.Scale(scheme="viridis", domain=[0, 100])),
    tooltip=[
        alt.Tooltip("gamma_center:O", title="γ"),
        alt.Tooltip("DNF_literals:O", title="# literals"),
        alt.Tooltip("max_mean_coop:Q", title="max mean coop (%)", format=".2f"),
        alt.Tooltip("best_variant:N", title="Best variant ID"),
        alt.Tooltip("n:Q", title="# variants in bin")
    ]
).properties(
    width=450, height=350,
    title=f"{chosen_norm} — Max *Mean* Cooperation by γ × DNF Complexity"
)

heat


alt.Chart(...)

In [69]:
import numpy as np
import pandas as pd
import altair as alt

# Import statsmodels for potential more advanced regression or to ensure required dependencies are available
# (though Altair's mark_regression handles the basic fit)
# import statsmodels.formula.api as smf # Not strictly needed for Altair's basic regression

# === Parameters ===
#chosen_base_norm = "SternJudging"    # 0) choose base social norm
fixed_gamma = 1                   # 1) fix gamma value

# === 0) Filter to chosen base norm ===
# NOTE: merged_df is assumed to be defined and loaded before this code block runs
# For a runnable example, let's assume merged_df is loaded here (e.g., from a CSV)
# For the purpose of adding the regression, we'll proceed assuming 'merged_df' exists.
#df = merged_df[merged_df["norm"] == chosen_base_norm].copy()
df = merged_df.copy()

# === ensure gamma column consistency & rounding ===
if "gamma_center" not in df.columns and "gamma_gaussian_n" in df.columns:
    df = df.rename(columns={"gamma_gaussian_n": "gamma_center"})

if "gamma_center" not in df.columns:
    raise KeyError("No gamma column found ('gamma_center' or 'gamma_gaussian_n').")

# round to 1 decimal to avoid float noise and keep only exact bin values
df["gamma_center"] = pd.to_numeric(df["gamma_center"], errors="coerce").round(1)
valid_gammas = np.round(np.arange(0, 1.01, 0.1), 1)

# filter rows to valid gammas first (drops messy intermediate values)
df = df[df["gamma_center"].isin(valid_gammas)]

# === 1) Filter to the fixed gamma value ===
df_gamma = df[np.isclose(df["gamma_center"], fixed_gamma)].copy()
if df_gamma.empty:
    raise ValueError(f"No rows found for gamma = {fixed_gamma}. Check rounding or available gamma values.")

# === 2) Compute mean cooperation per variant (averaging across runs) ===
# ensure average_cooperation is numeric
df_gamma["average_cooperation"] = pd.to_numeric(df_gamma["average_cooperation"], errors="coerce")
variant_avg = (
    df_gamma
    .groupby(["variant_id", "DNF_literals", "Emotion_Leniency"], as_index=False)
    .agg(mean_coop=("average_cooperation", "mean"),
          runs=("average_cooperation", "count"))    # how many runs contributed
)

# convert to percent if values are proportions in [0,1]
if variant_avg["mean_coop"].max() <= 1.0:
    variant_avg["mean_coop_pct"] = variant_avg["mean_coop"] * 100.0
else:
    variant_avg["mean_coop_pct"] = variant_avg["mean_coop"]

# === 3) For every DNF complexity, choose the variant with maximal mean cooperation ===
# use idxmax to get the variant row with the highest mean_coop
idx = variant_avg.groupby("DNF_literals")["mean_coop"].idxmax()
best_per_complexity = variant_avg.loc[idx].reset_index(drop=True)

# Optional: sort by complexity numeric ascending
best_per_complexity["DNF_literals"] = best_per_complexity["DNF_literals"].astype(int)
best_per_complexity = best_per_complexity.sort_values("DNF_literals")

# === ASSUMED best_per_complexity DATA STRUCTURE FOR REGRESSION ===
# best_per_complexity = pd.DataFrame({
#     'DNF_literals': [1, 2, 3, 4, 5],
#     'mean_coop_pct': [50.0, 65.0, 75.0, 70.0, 80.0],
#     'Emotion_Leniency': [0.1, 0.2, 0.3, 0.2, 0.4],
#     'variant_id': ['v1', 'v2', 'v3', 'v4', 'v5'],
#     'runs': [10, 10, 10, 10, 10]
# })

# === 4) Scatter plot: x = complexity, y = avg cooperation (max among variants per complexity) ===
# We must use :Q (Quantitative) for the x-axis for the regression to work.
x_encoding = alt.X("DNF_literals:Q", title="DNF complexity (number of literals)")
y_encoding = alt.Y("mean_coop_pct:Q", title="Max mean cooperation (%)",
                   scale=alt.Scale(domain=[0, 100]))

chart = alt.Chart(best_per_complexity).mark_circle(size=120).encode(
    x=x_encoding,
    y=y_encoding,
    color=alt.Color("Emotion_Leniency:O", title="Emotion Leniency",
                    scale=alt.Scale(scheme="viridis")),
    tooltip=[
        alt.Tooltip("DNF_literals:Q", title="DNF literals"),
        alt.Tooltip("variant_id:N", title="Variant ID"),
        alt.Tooltip("mean_coop_pct:Q", title="Mean coop (%)", format=".2f"),
        alt.Tooltip("runs:Q", title="# runs"),
        alt.Tooltip("Emotion_Leniency:O", title="Leniency")
    ]
).properties(
    width=600, height=350,
    title=f"Best variant per complexity at γ = {fixed_gamma}"
)

# === 5) Add Regression Line (Linear) - CORRECTED ===
# We use transform_regression() to calculate the line
# and then mark_line() to draw it.
regression_line = alt.Chart(best_per_complexity).transform_regression(
    'DNF_literals',        # The X variable
    'mean_coop_pct',       # The Y variable
    method='quad'        # Specify method (linear, poly, etc.)
).mark_line(
    color='red',           # Style the line
    strokeDash=[5, 5]      # Make it dashed
).encode(
    x=alt.X('DNF_literals:Q'),  # Re-encode X for the line layer
    y=alt.Y('mean_coop_pct:Q')  # Re-encode Y for the line layer
)

# Add text labels (optional) next to points showing variant short name
labels = alt.Chart(best_per_complexity).mark_text(dx=7, dy=0, align="left").encode(
    x=x_encoding,  # Use the quantitative encoding
    y=y_encoding,  # Use the quantitative encoding
    text=alt.Text("variant_id:N"),
)

# === 6) Combine the charts: Scatter + Regression Line + Labels ===
chart_with_regression = (chart + regression_line + labels).configure_title(fontSize=14)

chart_with_regression

alt.LayerChart(...)

In [40]:
import numpy as np
import pandas as pd
import altair as alt

# Import statsmodels for potential more advanced regression or to ensure required dependencies are available
# (though Altair's mark_regression handles the basic fit)
# import statsmodels.formula.api as smf # Not strictly needed for Altair's basic regression

# === Parameters ===
# chosen_base_norm = "SternJudging"    # 0) No longer filtering by a single norm
fixed_gamma = 0.8                      # 1) fix gamma value

# === 0) Filter to chosen base norm ===
# NOTE: merged_df is assumed to be defined and loaded before this code block runs
# We use the full merged_df, as requested.
df = merged_df.copy()

# === ensure gamma column consistency & rounding ===
if "gamma_center" not in df.columns and "gamma_gaussian_n" in df.columns:
    df = df.rename(columns={"gamma_gaussian_n": "gamma_center"})

if "gamma_center" not in df.columns:
    raise KeyError("No gamma column found ('gamma_center' or 'gamma_gaussian_n').")

# round to 1 decimal to avoid float noise and keep only exact bin values
df["gamma_center"] = pd.to_numeric(df["gamma_center"], errors="coerce").round(1)
valid_gammas = np.round(np.arange(0, 1.01, 0.1), 1)

# filter rows to valid gammas first (drops messy intermediate values)
df = df[df["gamma_center"].isin(valid_gammas)]

# === 1) Filter to the fixed gamma value ===
df_gamma = df[np.isclose(df["gamma_center"], fixed_gamma)].copy()
if df_gamma.empty:
    raise ValueError(f"No rows found for gamma = {fixed_gamma}. Check rounding or available gamma values.")

# === 2) Compute mean cooperation per variant (averaging across runs) ===
# We add "norm" to the groupby so we can distinguish them in the plot
df_gamma["average_cooperation"] = pd.to_numeric(df_gamma["average_cooperation"], errors="coerce")
variant_avg = (
    df_gamma
    .groupby(["norm", "variant_id", "DNF_literals", "Emotion_Leniency"], as_index=False)
    .agg(mean_coop=("average_cooperation", "mean"),
         runs=("average_cooperation", "count"))    # how many runs contributed
)

# convert to percent if values are proportions in [0,1]
if not variant_avg.empty and variant_avg["mean_coop"].max() <= 1.0:
    variant_avg["mean_coop_pct"] = variant_avg["mean_coop"] * 100.0
else:
    variant_avg["mean_coop_pct"] = variant_avg["mean_coop"]

# === 3) Use all averaged variants (no 'best of' filter) ===
# We are SKIPPING the step of choosing the maximal per complexity.
# We will plot all variants from variant_avg.

# Optional: ensure DNF_literals is int and sort
if not variant_avg.empty:
    variant_avg["DNF_literals"] = variant_avg["DNF_literals"].astype(int)
    variant_avg = variant_avg.sort_values("DNF_literals")
else:
    print(f"Warning: No data found after filtering for gamma = {fixed_gamma}. Chart will be empty.")


# === 4) Scatter plot: x = complexity, y = avg cooperation (plotting ALL variants) ===
# We must use :Q (Quantitative) for the x-axis for the regression to work.
x_encoding = alt.X("DNF_literals:Q", title="DNF complexity (number of literals)")
y_encoding = alt.Y("mean_coop_pct:Q", title="Mean cooperation (%)",
                   scale=alt.Scale(domain=[0, 100]))

chart = alt.Chart(variant_avg).mark_circle(size=80, opacity=0.7).encode(
    x=x_encoding,
    y=y_encoding,
    # Color by norm to distinguish the data points
    color=alt.Color("norm:N", title="Social Norm"),
    tooltip=[
        alt.Tooltip("norm:N", title="Norm"),
        alt.Tooltip("DNF_literals:Q", title="DNF literals"),
        alt.Tooltip("variant_id:N", title="Variant ID"),
        alt.Tooltip("mean_coop_pct:Q", title="Mean coop (%)", format=".2f"),
        alt.Tooltip("runs:Q", title="# runs"),
        alt.Tooltip("Emotion_Leniency:Q", title="Leniency")
    ]
).properties(
    width=600, height=350,
    title=f"All Norms — All variants at γ = {fixed_gamma}" # Updated title
).interactive() # Add interactive zoom/pan

# === 6) Combine the charts: Scatter + Regression Line ===
# The 'labels' chart was removed as it would be unreadable
chart_with_regression = (chart ).configure_title(fontSize=14)

chart_with_regression

alt.Chart(...)

# HEATMAPS
## Emotion Leniency

In [66]:
import pandas as pd
import numpy as np
import altair as alt

# === Parameters ===
chosen_norm = "SternJudging"  # Example

# === 1. Filter and unify gamma column name ===
norm_df = merged_df[merged_df.norm == chosen_norm].copy()

# Round γ and keep only clean bins (0.0, 0.1, ..., 1.0)
valid_gammas = np.round(np.arange(0, 1.01, 0.1), 1)
norm_df = norm_df[norm_df["gamma_center"].isin(valid_gammas)]

if "gamma_center" not in norm_df.columns and "gamma_gaussian_n" in norm_df.columns:
    norm_df = norm_df.rename(columns={"gamma_gaussian_n": "gamma_center"})

# Ensure numeric gamma
norm_df["gamma_center"] = pd.to_numeric(norm_df["gamma_center"], errors="coerce")

# === 2. Compute mean cooperation per variant first ===
variant_means = (
    norm_df.groupby(["variant_id", "gamma_center", "Emotion_Leniency"], as_index=False)
           .agg(mean_coop=("average_cooperation", "mean"),
                sd_coop=("average_cooperation", "std"),
                n_runs=("average_cooperation", "count"))
)

# Convert to % if needed
if variant_means["mean_coop"].max() <= 1:
    variant_means["mean_coop"] *= 100

# === 3. For each (γ, Leniency), find the variant with highest mean ===
idx = variant_means.groupby(["gamma_center", "Emotion_Leniency"])["mean_coop"].idxmax()
max_variants = variant_means.loc[idx].reset_index(drop=True)

# === 4. Plot heatmap ===
heat = alt.Chart(max_variants).mark_rect().encode(
    x=alt.X("gamma_center:O", title="γ"),
    y=alt.Y("Emotion_Leniency:O", title="Emotion Leniency", sort="descending"),
    color=alt.Color("mean_coop:Q", title="Max mean cooperation (%)",
                    scale=alt.Scale(scheme="viridis", domain=[0, 100])),
    tooltip=[
        alt.Tooltip("gamma_center:O", title="γ"),
        alt.Tooltip("Emotion_Leniency:O", title="Leniency"),
        alt.Tooltip("variant_id:N", title="Top Variant"),
        alt.Tooltip("mean_coop:Q", title="Mean coop (%)", format=".2f"),
        alt.Tooltip("sd_coop:Q", title="Std (%)", format=".2f"),
        alt.Tooltip("n_runs:Q", title="# runs")
    ]
).properties(
    width=400, height=350,
    title=f"{chosen_norm} — Maximal Cooperation by γ × Emotion Leniency"
)

heat


alt.Chart(...)

In [43]:
# pick your norm and gamma value
chosen_norm = "SternJudging"
gamma_value = 0.5  # 🔑 change this

# filter results for chosen norm and gamma
subset = merged_df[(merged_df["norm"] == chosen_norm) &
                   (merged_df["gamma_center"] == gamma_value)].copy()

# aggregate cooperation by (DNF_literals, Leniency)
agg = (
    subset.groupby(["DNF_literals", "Emotion_Leniency"], as_index=False)
          .agg(mean_coop=("average_cooperation", "mean"),
               std_coop=("average_cooperation", "std"),
               n=("variant_id", "nunique"))
)

# --- Scatterplot ---
chart = alt.Chart(agg).mark_circle(size=200).encode(
    x=alt.X("DNF_literals:Q", title="DNF complexity (# literals)"),
    y=alt.Y("Emotion_Leniency:Q", title="Emotion Leniency"),
    color=alt.Color("mean_coop:Q", title="Mean Cooperation",
                    scale=alt.Scale(scheme="viridis", domain=[0,100])),
    size=alt.Size("n:Q", title="# Variants"),
    tooltip=[
        "DNF_literals:Q",
        "Emotion_Leniency:Q",
        alt.Tooltip("mean_coop:Q", format=".2f", title="Mean coop"),
        alt.Tooltip("std_coop:Q", format=".2f", title="Std"),
        "n:Q"
    ]
).properties(
    title=f"{chosen_norm} — Cooperation by Complexity × Leniency (γ={gamma_value})",
    width=500, height=400
)

chart


alt.Chart(...)